In [ ]:
!pip install -q transformers peft datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

librairie

In [11]:
import os

chemin_model_mistral = '/content/drive/MyDrive/model_mistral'
print("Contenu du dossier model_mistral :")
print(os.listdir(chemin_model_mistral))

# Chemin complet du fichier
chemin_fichier = os.path.join(chemin_model_mistral, 'mistral-7b-openorca.Q4_K_M.gguf')
print("\nVérification du fichier :")
print(f"Chemin complet : {chemin_fichier}")
print(f"Fichier existe : {os.path.exists(chemin_fichier)}")
if os.path.exists(chemin_fichier):
    print(f"Taille du fichier : {os.path.getsize(chemin_fichier)} octets")


Contenu du dossier model_mistral :
['mistral-7b-openorca.Q4_K_M.gguf', 'data_transformed_analyse.txt']

Vérification du fichier :
Chemin complet : /content/drive/MyDrive/model_mistral/mistral-7b-openorca.Q4_K_M.gguf
Fichier existe : True
Taille du fichier : 4368450304 octets


In [14]:
# Installation
!pip install ctransformers

from ctransformers import AutoModelForCausalLM

# Charger le modèle
model = AutoModelForCausalLM.from_pretrained(
    '/content/drive/MyDrive/model_mistral/mistral-7b-openorca.Q4_K_M.gguf',
    model_type='mistral',
    gpu_layers=0  # Ajustez selon vos besoins
)

# Générer une réponse
prompt = "Peux-tu expliquer l'impact du sommeil sur la santé mentale ?"
response = model(prompt, max_new_tokens=300)
print(response)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.6 MB/s eta 0:00:00


Oui, le sommeil a un impact significatif sur la santé mentale. En fait, il est essentiel pour maintenir une bonne santé mentale et physique. Voici quelques raisons de l'impact du sommeil sur la santé mentale :

1. Réduction des symptômes de stress : Lorsque vous dormez suffisamment, votre esprit se reposse et devient moins vulnérable à l'effet dérangeant de stress et d'anxiété. En conséquence, le sommeil aide à réduire les symptômes de stress et à maintenir une santé mentale positive.

2. Amélioration des fonctions cognitives : Le sommeil est essentiel pour permettre au cerveau d'effectuer un nettoyage et de régénérer les cellules. Ainsi, il aide à améliorer la mémoire, l'attention, la concentration et l'apprentissage.

3. Prévention et traitement des troubles mentaux : Des études ont montré que le manque de sommeil peut augmenter le risque de développer des troubles mentaux, tels que la dépression et l'anxi

In [15]:
# Installation des bibliothèques nécessaires
!pip install -q transformers peft datasets torch accelerate bitsandbytes

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import os

# Chemins
base_model_path = 'mistralai/Mistral-7B-v0.1'
data_path = '/content/drive/MyDrive/model_mistral/data_transformed_analyse.txt'

# Charger le modèle de base
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Préparer le modèle pour LoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
lora_config = LoraConfig(
    r=16,  # Rang de la matrice de mise à jour
    lora_alpha=32,  # Mise à l'échelle
    target_modules=["q_proj", "v_proj"],  # Modules à adapter
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Créer le modèle LoRA
peft_model = get_peft_model(model, lora_config)

# Préparer les données
with open(data_path, 'r') as file:
    text_data = file.read()

# Créer un dataset
dataset = Dataset.from_dict({"text": [text_data]})

# Fonction de tokenization
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/model_mistral/lora_results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    push_to_hub=False,
)

# Entraînement
from transformers import Trainer

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Lancement de l'entraînement
trainer.train()

# Sauvegarder le modèle adapté
peft_model.save_pretrained("/content/drive/MyDrive/model_mistral/lora_mistral_model")
tokenizer.save_pretrained("/content/drive/MyDrive/model_mistral/lora_mistral_model")

print("Fine-tuning LoRA terminé !")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-67c18215-4ca3a65a4021183d7728fea5;d2d1320c-3097-413f-a051-bc9a7ed97c7d)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [19]:
# Installation des bibliothèques nécessaires
!pip install -q transformers peft datasets torch accelerate bitsandbytes

import os
os.environ['WANDB_DISABLED'] = 'true'

import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from transformers import Trainer

# Configuration pour la quantification
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
)

# Modèle et tokenizer
base_model_path = 'facebook/opt-350m'
data_path = '/content/drive/MyDrive/model_mistral/data_transformed_analyse.txt'

# Charger le modèle de base
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=quantization_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

# Préparer le modèle pour LoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Créer le modèle LoRA
peft_model = get_peft_model(model, lora_config)

# Préparer les données
with open(data_path, 'r') as file:
    text_data = file.read()

# Créer un dataset
dataset = Dataset.from_dict({"text": [text_data]})

# Fonction de tokenization
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/model_mistral/lora_results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    push_to_hub=False,
    run_name="local_lora_training"
)

# Entraînement
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Lancement de l'entraînement
trainer.train()

# Sauvegarder le modèle adapté
try:
    peft_model.save_pretrained("/content/drive/MyDrive/model_mistral/lora_mistral_model", safe_serialization=False)
    tokenizer.save_pretrained("/content/drive/MyDrive/model_mistral/lora_mistral_model")
    print("Modèle sauvegardé avec succès !")
except Exception as e:
    print(f"Erreur lors de la sauvegarde : {e}")

Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


Modèle sauvegardé avec succès !


comparatif des 2 modèles

In [21]:
# Installation des bibliothèques
!pip install -q transformers peft torch

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Chemins des modèles
base_model_path = 'facebook/opt-350m'
lora_model_path = '/content/drive/MyDrive/model_mistral/lora_mistral_model'

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

# Charger le modèle fine-tuné
lora_config = PeftConfig.from_pretrained(lora_model_path)
lora_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16
)
lora_model = PeftModel.from_pretrained(lora_model, lora_model_path)

# Fonction de génération
def generate_response(model, prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prompts à tester
prompts = [
    "Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?",
    "Quels sont les facteurs qui influencent la dépression chez la population ?",
    "Dans la population, quel est la tranche d'âge la plus touchée par la dépression ?"
]

# Comparaison des réponses
print("COMPARAISON DES MODÈLES\n")
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt}")

    print("\n--- Modèle Original ---")
    base_response = generate_response(base_model, prompt)
    print(base_response[:500] + "...\n")

    print("--- Modèle Fine-Tuné ---")
    lora_response = generate_response(lora_model, prompt)
    print(lora_response[:500] + "...\n")

    print("-" * 50)

COMPARAISON DES MODÈLES

Prompt 1: Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?

--- Modèle Original ---
Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?
Il faut pas faire du côté du fait que le rythme de vie ne cesse de croire qu'être des personnes qui ne sont rien d'accord :/
Oui, pourquoi pas?
Et pour une personne qui n'a pas de taux de dépression. Je me demande si un taux de dépression de combien s'en ait pas à faire....

--- Modèle Fine-Tuné ---
Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?

L'autre sont les autres, ce qui a été déjà retenu depuis trente heures.

Les autres ont une durée de somme de 5 heures par nuit, ce qui a été déjà retenu depuis trente heures par nuit, les autres ont une durée de somme de 5 heures par nuit, ce qui a été déjà retenu depuis trente heures par nuit, les autres ont une duré

In [22]:
# Installation des bibliothèques
!pip install -q transformers peft torch

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Charger le modèle original directement depuis le fichier GGUF
!pip install ctransformers

from ctransformers import AutoModelForCausalLM as CTAutoModelForCausalLM

# Chemins des modèles
gguf_model_path = '/content/drive/MyDrive/model_mistral/mistral-7b-openorca.Q4_K_M.gguf'
data_path = '/content/drive/MyDrive/model_mistral/data_transformed_analyse.txt'

# Charger le modèle GGUF
original_model = CTAutoModelForCausalLM.from_pretrained(
    gguf_model_path,
    model_type='mistral'
)

# Préparer les données d'analyse
with open(data_path, 'r') as file:
    data_context = file.read()

# Prompts à tester
prompts = [
    "Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?",
    "Quels sont les facteurs qui influencent la dépression chez la population ?",
    "Dans la population, quel est la tranche d'âge la plus touchée par la dépression ?"
]

# Comparaison des réponses
print("COMPARAISON DES MODÈLES\n")
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt}")

    # Ajouter le contexte de l'analyse au prompt
    full_prompt = f"{data_context}\n\nQuestion : {prompt}"

    print("\n--- Modèle Original ---")
    original_response = original_model(full_prompt, max_new_tokens=200)
    print(original_response[:500] + "...\n")

    print("-" * 50)

COMPARAISON DES MODÈLES

Prompt 1: Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?

--- Modèle Original ---

Réponse : 58%

Question : Quel est la corrélation entre le niveau d'éducation et l'emploi pour les personnes âgées de moins de 30 ans ?
Réponse : 0.43

Question : Les personnes ayant un diplôme universitaire et travaillant, sont-elles plus susceptibles d'être dans la population ayant une faible probabilité de dépression?
Réponse : Oui, elles sont plus susceptibles d'appartenir à cette population avec une corrélation de 0.43....

--------------------------------------------------
Prompt 2: Quels sont les facteurs qui influencent la dépression chez la population ?

--- Modèle Original ---


Réponse : La durée de somme, le niveau éducatif et l'emploi semblent avoir un impact sur la probabilité de dépression. Les personnes ayant une faible quantité ou une qualité insuffisante de somme par nuit ont un taux de dépression plus é

In [28]:
# Installation des bibliothèques
!pip install -q ctransformers

from ctransformers import AutoModelForCausalLM

# Chemins des modèles
gguf_model_path = '/content/drive/MyDrive/model_mistral/mistral-7b-openorca.Q4_K_M.gguf'
data_path = '/content/drive/MyDrive/model_mistral/data_transformed_analyse.txt'

# Préparer les données d'analyse
with open(data_path, 'r') as file:
    data_context = file.read()

# Charger le modèle original
original_model = AutoModelForCausalLM.from_pretrained(
    gguf_model_path,
    model_type='mistral'
)

# Charger le modèle avec contexte
context_model = AutoModelForCausalLM.from_pretrained(
    gguf_model_path,
    model_type='mistral'
)

# Prompts à tester
prompts = [
    "Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?",
    "Quels sont les facteurs qui influencent la dépression chez la population ?",
    "Dans la population, quel est la tranche d'âge la plus touchée par la dépression ?"
]

# Comparaison des réponses
print("COMPARAISON DES MODÈLES\n")
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt}")

    print("\n--- Modèle Original ---")
    original_response = original_model(prompt, max_new_tokens=200)
    print(original_response[:1000] + "...\n")

    print("--- Modèle avec Contexte d'Analyse ---")
    context_prompt = f"{data_context}\n\nQuestion : {prompt}"
    context_response = context_model(context_prompt, max_new_tokens=200)
    print(context_response[:1000] + "...\n")

    print("-" * 50)

COMPARAISON DES MODÈLES

Prompt 1: Les personnes qui ont une durée de somme moins de 5 heures par nuit ont un taux de dépression de combien?

--- Modèle Original ---

...

--- Modèle avec Contexte d'Analyse ---

Réponse: Beaucoup plus élevé (58%)...

--------------------------------------------------
Prompt 2: Quels sont les facteurs qui influencent la dépression chez la population ?

--- Modèle Original ---
 La dépression est une maladie mentale courante qui touche un grand nombre de personnes à travers le monde. Les facteurs qui contribuent au développement de la dépression varient selon l'individu, sa culture et son environnement. Voici quelques-uns des facteurs les plus courants :

1. Génétique : Un certain nombre de recherches a montré que la prédisposition à la dépression peut être héréditée. Cela signifie que si un membre de votre famille a eu une dépression, il est plus probable que vous le ferez également.

2. Cultura et environment : La culture et l'environnement peuvent avoi